In [1]:
import numpy as np

In [2]:
# Ejemplo - Construcción de grilla

class Mapa:

    def __init__(self,ancho,alto,inicio):
        self.ancho = ancho
        self.alto = alto
        self.fila = inicio[0]
        self.columna = inicio[1]

    def conjunto(self,recompensas,acciones):
        self.recompensas = recompensas # diccionario de posibles recompensas para cada estado (fila,columna) : r
        self.acciones = acciones # diccionario de posibles acciones para cada estado (fila,columna) : ('U','D','L','R')

    def fijar_estado(self,s):
        self.fila = s[0]
        self.columna = s[1]

    def estado_actual(self):
        return (self.fila,self.columna)

    def es_final(self,s):
        return s not in self.acciones

    def mover(self,accion):
        if accion in self.acciones[self.estado_actual()] :
            if accion == 'U':
                self.fila -= 1
            elif accion == 'D':
                self.fila += 1
            elif accion == 'L':
                self.columna -= 1
            elif accion == 'R':
                self.columna += 1
        return self.recompensas.get(self.estado_actual(),0)

    def deshacer(self,accion):
        if accion in self.acciones[self.estado_actual()] :
            if accion == 'U':
                self.fila += 1
            elif accion == 'D':
                self.fila -= 1
            elif accion == 'L':
                self.columna += 1
            elif accion == 'R':
                self.columna -= 1
        assert(self.estado_actual() in self.todos_estados())

    def fin(self):
        return self.estado_actual not in self.acciones

    def todos_estados(self):
        return set(list(self.acciones.keys()) + list(self.recompensas.keys()))

In [3]:
# Ejemplo - Configuración del problema

def mapa_con_costo(costo_paso = -0.1):
    g = Mapa(4,3,(2,0)) # ancho de 4 celdas, alto de 3 celdas y posición inicial en (2,0)
    recompensas = {(0,3) : 1, (1,3):-1}
    acciones = {
        (0,0) : ('D','R'),
        (0,1) : ('L','R'),
        (0,2) : ('L','R','D'),
#        (0,3) : ('',),
        (1,0) : ('U','D'),
#        (1,1) : ('',),
        (1,2) : ('U','D','R'),
#        (1,3) : ('',),
        (2,0) : ('U','R'),
        (2,1) : ('L','R'),
        (2,2) : ('U','L','R'),
        (2,3) : ('L','U')
        }
    g.conjunto(recompensas,acciones)
    recompensas = {}
    for fila in range(g.alto):
        for columna in range(g.ancho):
            recompensas[(fila,columna)] = costo_paso
    recompensas[(0,3)] = 1
    recompensas[(1,3)] = -1
    recompensas.pop((1,1),None)
    g.conjunto(recompensas,g.acciones)
    return g

In [4]:
# Ejemplo - Mostrar valores
def mostrar_valores(V,g):
    for fila in range(g.alto):
        print(20*"-")
        for columna in range(g.ancho):
            v = V.get((fila,columna),0)

            #if (v >= 0) :
            print(" %.2f|" % v, end='')
            #else :
             #   print "%.2f|"  % v,
        print("")

def mostrar_politica(P,g) :
    for fila in range(g.alto):
        print(20*"-")
        for columna in range(g.ancho):
            a = P.get((fila,columna),'')
            print(" %s |" % a, end='')
        print("")

In [5]:
# Ejemplo - Policy Iteration (configuración)
gamma = 0.9
epsilon = 1e-3
total_acciones = ('U','D','L','R')
mapa = mapa_con_costo()
estados = mapa.todos_estados()
V = {}
pi = {}
for fila in range(mapa.alto):
    for columna in range(mapa.ancho):
        V[(fila,columna)] = 0.0

for s in mapa.acciones.keys():  # Para todos los estados se inicializa una politica aleatoria
    pi[s] = np.random.choice(total_acciones)

In [6]:
# Ejemplo - Policy Iteration
repeticiones = 10
iteraciones = 0
while True:
    iteraciones += 1
# Evaluación de política
    while True :
        delta = 0
        v_anterior = 0.0
        for s in estados:
            if s not in pi:
                continue
            mapa.fijar_estado(s)
            v_anterior = V[s]
            accion = pi[s]
            r = mapa.mover(accion)
            nuevo_estado = mapa.estado_actual()
            V[s] = r + gamma * V[nuevo_estado]
            delta = max(delta,abs(v_anterior - V[s]))
        if delta < epsilon :
            break
# Mejora de politica
    politica_estable = True
    for s in estados:
        if s not in pi:
            continue
        accion_anterior = pi[s]
        max_val = float("-inf")
        for accion in mapa.acciones[s]:
            mapa.fijar_estado(s)
            r = mapa.mover(accion)
            nuevo_estado = mapa.estado_actual()
            val = r + gamma * V[nuevo_estado]
            if val > max_val :
                max_val = val
                max_accion = accion
        pi[s] = max_accion
        if accion_anterior != pi[s] :
            politica_estable = False
    if politica_estable == True or iteraciones >= repeticiones:
        mostrar_politica(pi,mapa)
        break

--------------------
 R | R | R |  |
--------------------
 U |  | U |  |
--------------------
 U | R | U | L |


In [7]:
# Ejemplo - Q-learning
# OJO con exploración - explotación : este agente se estanca en un óptimo local
repeticiones = 200
inicio = (2,0)
alpha = 0.8
gamma = 0.9
total_acciones = ('U','D','L','R')
mapa = mapa_con_costo()
estados = mapa.todos_estados()
Q = {}
pi = {}
timeout = 100
for s in estados:
    for a in total_acciones:
        Q[(s,a)] = 0.0

s = inicio
for n in range(repeticiones):
    k = 0
    mapa.fijar_estado(inicio)
#    print("inicio",inicio, mapa.es_final(inicio))
    while not mapa.es_final(s) and k < timeout:
#      print("episodio",n,"paso",k)
      s = mapa.estado_actual()
      r = mapa.recompensas[s]
      aux = 0
      for accion_a_ejecutar in mapa.acciones[s]:

        if Q[(s,accion_a_ejecutar)] >= aux:
          aux = Q[(s,accion_a_ejecutar)]
          a = accion_a_ejecutar
      mayorQ = -float("inf")
      for nueva_accion in mapa.acciones[s]:
        mapa.mover(nueva_accion)
        sn = mapa.estado_actual()
        if Q[(sn,nueva_accion)] > mayorQ:
          mayorQ = Q[(sn,nueva_accion)]
        mapa.deshacer(nueva_accion)
      Q[(s,a)] += alpha*(r + gamma*mayorQ - Q[(s,a)])
      mapa.mover(a)
      pi[s] = a
      k += 1
mostrar_politica(pi,mapa)
#print(Q)

--------------------
 D | L |  |  |
--------------------
 D |  |  |  |
--------------------
 D | L |  |  |
